<a href="https://colab.research.google.com/github/fatmaaerr/AMD_Ukraine_Conflict_Market_Basket_A/blob/main/FP_Growth_Algorithm_MDA_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install kaggle spark-nlp==3.4.2 pyspark==3.2.0 ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 142 kB 10.9 MB/s 
     |████████████████████████████████| 281.3 MB 36 kB/s 
     |████████████████████████████████| 198 kB 67.7 MB/s 
     |████████████████████████████████| 1.6 MB 44.1 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805913 sha256=eb2c40eac3eb510e7ef5a584c2f5f9b457679249d92325a45acbddc6f0e1bd88
  Stored in directory: /root/.cache/pip/wheels/23/f6/d3/110e53bd43baeb8d7d38049733d48e39cbecd056f01dba7ee8
Successfully built pyspark
time: 453 µs (started: 2022-12-08 22:05:02 +00:00)


In [2]:
from pyspark.sql import SparkSession
import os
from glob import glob
from pyspark.sql.types import FloatType
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import * 
import string
from pyspark.sql import functions as F
from pyspark.ml.fpm import FPGrowth
import pandas as pd
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import udf
from itertools import combinations
import math
import numpy as np
import logging
from joblib import Parallel, delayed, cpu_count
from collections import namedtuple
from itertools import product
import time
from itertools import chain
spark = sparknlp.start(spark32 = True)
sc = spark.sparkContext

time: 46.7 s (started: 2022-12-08 22:05:12 +00:00)


In [3]:
import matplotlib.pyplot as plt
from tqdm import tqdm
import csv
import os

time: 228 ms (started: 2022-12-08 22:06:31 +00:00)


In [4]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark3.0.2
!wget -q http://apache.osuosl.org/spark/spark-3.0.3/spark-3.0.3-bin-hadoop3.2.tgz

# unzip it
!tar xf spark-3.0.3-bin-hadoop3.2.tgz
# install findspark 
!pip install -q findspark

tar: spark-3.0.3-bin-hadoop3.2.tgz: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now
time: 18.3 s (started: 2022-12-08 22:06:35 +00:00)


In [5]:
# Kaggle API to download the dataset
import os
kaggle_api = {"username":"fatmaer","key":"02db3447e0d87603a7e4cb10816fe1b7"}
os.environ['KAGGLE_USERNAME'] = kaggle_api['username']
os.environ['KAGGLE_KEY'] = kaggle_api['key']

time: 847 µs (started: 2022-12-08 22:07:01 +00:00)


In [6]:
# Kaggle authentication and download
import kaggle
kaggle.api.authenticate()
!  kaggle datasets download BWANDOWANDO/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 13.1G/13.1G [07:40<00:00, 20.8MB/s]
100% 13.1G/13.1G [07:40<00:00, 30.6MB/s]
time: 7min 42s (started: 2022-12-08 22:07:01 +00:00)


In [7]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0903_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

In [31]:
# Importing data , analysis and visualization

time: 1.2 ms (started: 2022-12-09 00:40:14 +00:00)


In [8]:
read_file = ["UkraineWar/UkraineWar/UkraineCombinedTweetsDeduped_MAR10.csv.gzip"]


time: 857 µs (started: 2022-12-08 22:17:01 +00:00)


In [9]:
collect_file = []
columns = ["userid","text","language",'retweetcount']
for data in read_file:

  df = pd.read_csv(data, compression='gzip', index_col=0,encoding='utf-8', quoting=csv.QUOTE_ALL, low_memory=False)
  df= df[columns]
  collect_file.append(df)
  print(f"The data {data} has {len(df)} number of rows ")

The data UkraineWar/UkraineWar/UkraineCombinedTweetsDeduped_MAR10.csv.gzip has 460075 number of rows 
time: 7.83 s (started: 2022-12-08 22:17:01 +00:00)


In [10]:
df_combined = pd.concat(collect_file, axis=0)
df_combined.shape

(460075, 4)

time: 22 ms (started: 2022-12-08 22:17:09 +00:00)


In [11]:
df_combined

,userid,text,language,retweetcount
4557550,382303683,This #Russia-dropped bomb would flatten a buil...,en,6572
4557551,1433449200470032385,#Zelensky:\n\nThere was little Bandera there? ...,en,25
4557552,3689308343,@SecBlinken If the horrific attacks on #Ukrain...,en,1
4557553,567289542,Thousands of residents from the northeastern #...,en,0
4557554,1276373615466799104,"In 2 hours, we are #streaming 3 #indiegames fr...",en,1
...,...,...,...,...
5017620,46023832,"'Fading' \n\n""War is only a cowardly escape fr...",en,9
5017621,27004493,#UKRAINE WHAT IS USA doing to take in refugee ...,en,0
5017622,2612775241,"🔴 Due to russian aggression against #Ukraine, ...",en,75
5017623,867067128,ULTIMA HORA: #Rusia acaba de atacar el Institu...,es,596


time: 26.2 ms (started: 2022-12-08 22:17:28 +00:00)


In [24]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 460075 entries, 4557550 to 5017624
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   userid        460075 non-null  int64 
 1   text          460075 non-null  object
 2   language      460075 non-null  object
 3   retweetcount  460075 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 17.6+ MB
time: 98.2 ms (started: 2022-12-07 11:29:06 +00:00)


In [25]:
df_combined.isnull().sum()

userid          0
text            0
language        0
retweetcount    0
dtype: int64

time: 82.8 ms (started: 2022-12-07 11:29:12 +00:00)


In [26]:
df_combined["language"].value_counts().head(10)

en     303769
fr      26510
de      23488
it      23256
es      20751
und     19080
pl       5799
tr       5417
th       4368
ja       4288
Name: language, dtype: int64

time: 30.1 ms (started: 2022-12-07 11:29:14 +00:00)


In [12]:
df =  df_combined[df_combined["language"] == "en"]
df.shape

(303769, 4)

time: 76.4 ms (started: 2022-12-08 22:17:35 +00:00)


In [13]:
df.drop_duplicates(subset=['text'],keep='last',inplace=True)
df.shape

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


(92842, 4)

time: 247 ms (started: 2022-12-08 22:17:36 +00:00)


In [14]:
sorted_tweets = df[['userid', 'text', "language",'retweetcount' ]].sort_values(by = 'retweetcount', ascending=False).head(10)

time: 25.7 ms (started: 2022-12-08 22:17:38 +00:00)


In [30]:
sorted_tweets

,userid,text,language,retweetcount
4988826,562022972,.@ZelenskyyUa's tv address to the Russian (!) ...,en,147590
4821262,1454609345723912193,My daughter and I surviving the night in Ukrai...,en,107674
5016654,816720226821738496,"This, out of #Ukraine, is 100% one of the most...",en,103340
5017618,397469365,"Little girl singing ""Let it go"" in a shelter\n...",en,74898
5013514,4615594696,JUST IN: #Russian state TV channels have been ...,en,60979
5013447,710150049389551617,The Anonymous collective is officially in cybe...,en,53574
5014586,566300853,"In #StPetersburg, #Russia, the police detained...",en,48602
4676102,175074642,"""It is not the job of black people &amp; ethni...",en,47720
4831684,1012381844430577664,I learn to use #Kalashnikov and prepare to bea...,en,46523
5017092,1017922214539087874,#Anonymous message to Vladimir Putin https://t...,en,44583


time: 13.1 ms (started: 2022-12-07 11:29:22 +00:00)


Using this kind of algoritm with text really difiicult for processing. We can delete Retwitts less than 2.Aim is here understand tren situation for this topic

```
# This is formatted as code
```



In [15]:
spark = SparkSession.builder.getOrCreate()
columns = ["userid","text","language","retweetcount"]
pysparkDF = spark.createDataFrame(data = df, schema = columns)
pysparkDF.printSchema()
pysparkDF.show(truncate=False)

root
 |-- userid: long (nullable = true)
 |-- text: string (nullable = true)
 |-- language: string (nullable = true)
 |-- retweetcount: long (nullable = true)

+-------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+------------+
|userid             |text                                                                                                                                                                                                                                                                                                        |language|retweetcount|
+-------------------+---------------------------------------------------------------------------------------------------------------------------------

In [32]:
pysparkDF.show(5)

+-------------------+--------------------+--------+------------+
|             userid|                text|language|retweetcount|
+-------------------+--------------------+--------+------------+
|1421921881528016896|https://t.co/bBIa...|      en|           0|
| 857684571537760260|"One of the first...|      en|           0|
|         2190329623|Older people make...|      en|           0|
|           41670073|Rabobank: #Ukrain...|      en|           0|
|           49475701|Tourism industrie...|      en|           0|
+-------------------+--------------------+--------+------------+
only showing top 5 rows

time: 549 ms (started: 2022-12-07 11:30:07 +00:00)


In [17]:

pysparkDF2=pysparkDF


time: 1.27 ms (started: 2022-12-08 22:18:09 +00:00)


In [18]:
pysparkDF2 = pysparkDF2.withColumn("retweetcount", pysparkDF2["retweetcount"].cast(FloatType()))
only_text = (
            pysparkDF2
            .dropna(subset = ['text','language'])
            .filter((pysparkDF2.language == "en") & (pysparkDF2.retweetcount >= pysparkDF2.approxQuantile("retweetcount", [0.5], 0.25)[0])) 
            .dropDuplicates(["userid","text"])
            .select("text")
            .sample(fraction = 0.010,seed = 3000)) # Change the Fraction Argument to take different sample sizes

only_text.show(5)

+--------------------+
|                text|
+--------------------+
|@BorisJohnson Unf...|
|Raise your hand i...|
|Brutal beating of...|
|@DmytroKuleba @ua...|
|@nytimes Lynsey i...|
+--------------------+
only showing top 5 rows

time: 6.69 s (started: 2022-12-08 22:18:12 +00:00)


In [19]:

# Get row count
rows = only_text.count()

# Get columns count
cols = len(only_text.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")

DataFrame Dimensions : (900, 1)
DataFrame Rows count : 900
DataFrame Columns count : 1
time: 2.97 s (started: 2022-12-08 22:18:22 +00:00)


In [20]:
only_text.write.csv("Sample_text")

time: 2.3 s (started: 2022-12-08 22:18:32 +00:00)


In [ ]:
#Text preprocessing

In [21]:
# Download lemmatizer dependencies
!wget -q https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

# Loading 'text' data into spark NLP  
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Tokenization: Converting single words into individual tokens
tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

# Normaliszation: Removing numbers, punctuations and other non-alphabet characters
normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(True) # remove punctuations (keep alphanumeric chars)
    # if we don't set CleanupPatterns, it will only keep alphabet letters ([^A-Za-z])

# Lemmatization: Converting the words to their root forms
lemmatizer = Lemmatizer() \
    .setInputCols(["normalized"]) \
    .setOutputCol("lemma") \
    .setDictionary("AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

# Clean Stopwords: Removing the stopwords which are by-default loaded in the stopwords cleaner library
stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("lemma")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)
      # .setStopWords(["no", "without"])
stopwordsList = stopwords_cleaner.getStopWords()
custom_stopword_list = ['ukraine','russia','ukrainian','russian','chinese'] # add a list of customized stop words here
stopwordsList.extend(custom_stopword_list)
stopwords_cleaner = stopwords_cleaner.setStopWords(stopwordsList)

# Generating ngrams: Identifying a set of words ('n' no of words) used together
ngrams_cum = NGramGenerator().setInputCols(["cleanTokens"]) \
      .setOutputCol("ngrams") \
      .setN(1)\
      .setEnableCumulative(False)\
      .setDelimiter("_") 

# Providing the output in the finished format
finisher = Finisher() \
    .setInputCols(["ngrams"]) \
    .setIncludeMetadata(False)

# Setting up the sequence of execution
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    tokenizer,
    normalizer,
    lemmatizer,
    stopwords_cleaner,
    ngrams_cum,
    finisher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipelineModel = nlpPipeline.fit(empty_df)
result = pipelineModel.transform(only_text)

result = result.withColumn("unique", F.array_distinct("finished_ngrams"))

# Filtering data based on word count (word count should be greated than 1 and less than 75% of distribution (to avoid outlier and extremly lenghthy tweets))

slen = udf(lambda s: len(s), IntegerType())
result = result.withColumn("word_count", slen(result.unique))
result = result.filter((result.word_count > 1) & (result.word_count < result.approxQuantile("word_count", [0.75], 0.25)[0]))
result.show(5)
transactionList = result.select('unique').rdd.flatMap(lambda x: x)

+--------------------+--------------------+--------------------+----------+
|                text|     finished_ngrams|              unique|word_count|
+--------------------+--------------------+--------------------+----------+
|@BorisJohnson Unf...|[borisjohnson, un...|[borisjohnson, un...|         9|
|Raise your hand i...|[raise, hand, wan...|[raise, hand, wan...|        15|
|Brutal beating of...|[brutal, beat, pr...|[brutal, beat, pr...|        13|
|@DmytroKuleba @ua...|[dmytrokuleba, ua...|[dmytrokuleba, ua...|        25|
|@nytimes Lynsey i...|[nytimes, lynsey,...|[nytimes, lynsey,...|        19|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows

time: 15.5 s (started: 2022-12-08 22:18:37 +00:00)


In [22]:

# Get row count
rows = result.count()

# Get columns count
cols = len(result.columns)

print(f"DataFrame Dimensions : {(rows,cols)}")
print(f"DataFrame Rows count : {rows}")
print(f"DataFrame Columns count : {cols}")

DataFrame Dimensions : (899, 4)
DataFrame Rows count : 899
DataFrame Columns count : 4
time: 2.94 s (started: 2022-12-08 22:19:14 +00:00)


transactionList = result.select('unique').rdd.flatMap(lambda x: x)

In [23]:
print(transactionList)

PythonRDD[77] at RDD at PythonRDD.scala:53
time: 21.4 ms (started: 2022-12-08 22:19:24 +00:00)


In [24]:
print( transactionList.collect() )

[['borisjohnson', 'unfortunately', 'boris', 'government', 'friend', 'ally', 'greatbritain', 'europe', 'witness'], ['raise', 'hand', 'want', 'biden', 'nato', 'stand', 'putin', 'establish', 'humanitarian', 'fly', 'zone', 'istandwithukraine', 'stoprussia', 'eu', 'russiaukrainewar'], ['brutal', 'beat', 'protest', 'woman', 'police', 'man', 'either', 'work', 'army', 'film', 'course', 'shame', 'httpstcohuxrgwlkps'], ['dmytrokuleba', 'uaparliament', 'europe', 'amp', 'nato', 'close', 'sky', 'hear', 'many', 'innocent', 'life', 'child', 'give', 'away', 'war', 'emmanuelmacron', 'vonderleyen', 'eucopresident', 'epedinburgh', 'eucommission', 'europarlen', 'un', 'potus', 'olafscholz', 'borisjohnson'], ['nytimes', 'lynsey', 'amazing', 'shes', 'kind', 'person', 'legend', 'make', 'love', 'feel', 'horrible', 'family', 'kill', 'putin', 'must', 'pay', 'standwithukraine', 'stopputin', 'ukrainewar'], ['right', 'far', 'tactical', 'hit', 'sec', 'tiktok', 'ukrainewar', 'istandwithukraine', 'httpstcoweuuwmzszb']

In [25]:
transactionList = result.select('unique').rdd.flatMap(lambda x: x)

time: 1.03 s (started: 2022-12-08 22:19:32 +00:00)


In [157]:
result.summary().show(5)

+-------+--------------------+------------------+
|summary|                text|        word_count|
+-------+--------------------+------------------+
|  count|                 899|               899|
|   mean|                null|15.681868743047831|
| stddev|                null| 6.848586287921675|
|    min|"#Ukraine has acc...|                 2|
|    25%|                null|                11|
+-------+--------------------+------------------+
only showing top 5 rows

time: 4.18 s (started: 2022-12-07 16:59:58 +00:00)


In [55]:
#FPGrowth Algorithm

time: 526 µs (started: 2022-12-07 13:12:10 +00:00)


In [26]:
np.random.seed(1)
# Try different thresholds for minimum support 
th_res = [0.01,0.02,0.06,0.09,0.1]
fpgrowth_time_res = {}

for th in tqdm( th_res ):

  fpGrowth = FPGrowth(itemsCol="unique", minSupport = th)

  # compute time difference
  start_time = time.time()
  model = fpGrowth.fit(result)

  fpgrowth_time_res[th] = time.time() - start_time

100%|██████████| 5/5 [00:23<00:00,  4.77s/it]

time: 23.9 s (started: 2022-12-08 22:19:47 +00:00)


In [27]:
fpgrowth_time_res

{0.01: 5.544050216674805,
 0.02: 3.6469178199768066,
 0.06: 3.9798896312713623,
 0.09: 7.0657079219818115,
 0.1: 3.5440526008605957}

time: 4.79 ms (started: 2022-12-08 22:20:16 +00:00)


https://towardsdatascience.com/market-basket-analysis-using-pysparks-fpgrowth-55c37ebd95c0

https://www.kaggle.com/code/megan3/market-basket-analysis-using-pyspark

minSupport=0.02, minConfidence=0.02

In [28]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets using support, lift, and confidence.
fpGrowth = FPGrowth(itemsCol="unique", minSupport=0.02, minConfidence=0.02)
model = fpGrowth.fit(result)
# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the consequents as prediction
model.transform(result).show()
transformed = model.transform(result)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+--------------------+----+
|               items|freq|
+--------------------+----+
|          [sanction]|  19|
|             [fight]|  19|
|           [refugee]|  25|
|              [cant]|  22|
|               [see]|  32|
|              [even]|  23|
|        [stoprussia]|  27|
|  [russiaukrainewar]|  57|
|            [please]|  35|
|              [look]|  21|
|             [money]|  18|
|              [last]|  21|
|             [every]|  24|
|              [need]|  29|
|              [take]|  40|
|  [standwithukraine]| 139|
|[standwithukraine...|  23|
|[standwithukraine...|  21|
|          [mariupol]|  50|
|            [attack]|  26|
+--------------------+----+
only showing top 20 rows

+------------------+-------------------+-------------------+------------------+--------------------+
|        antecedent|         consequent|         confidence|              lift|             support|
+------------------+-------------------+-------------------+------------------+--------------------+

minSupport=0.006, minConfidence=0.006

In [30]:
#Frequent Pattern Growth – FP Growth is a method of mining frequent itemsets using support, lift, and confidence.
fpGrowth = FPGrowth(itemsCol="unique", minSupport=0.006, minConfidence=0.006)
model = fpGrowth.fit(result)
# Display frequent itemsets.
model.freqItemsets.show()
items = model.freqItemsets
# Display generated association rules.
model.associationRules.show()
rules = model.associationRules
# transform examines the input items against all the association rules and summarize the consequents as prediction
model.transform(result).show()
transformed = model.transform(result)

+--------------------+----+
|               items|freq|
+--------------------+----+
|            [leader]|  14|
|     [leader, putin]|   7|
|        [aggression]|  10|
|          [sanction]|  19|
|     [sanction, war]|   6|
|             [large]|   6|
|             [break]|  11|
|            [little]|  11|
|               [hit]|   9|
|              [book]|   6|
|[stoprussianaggre...|  13|
|[stoprussianaggre...|   7|
|                [uk]|  17|
|[closetheskyoveru...|   6|
|               [sky]|  16|
| [sky, borisjohnson]|   7|
|[sky, borisjohnso...|   6|
|[sky, borisjohnso...|   6|
|[sky, borisjohnso...|   7|
|[sky, borisjohnso...|   7|
+--------------------+----+
only showing top 20 rows

+--------------------+----------------+------------------+------------------+--------------------+
|          antecedent|      consequent|        confidence|              lift|             support|
+--------------------+----------------+------------------+------------------+--------------------+
|[epe

In [158]:
words_basket = result.select('unique').collect()


list_words_baskets = [ row[0] for row in words_basket]

# Save the actor baskets list
f = open('list_words_baskets.txt','w')
for el in list_words_baskets:
    f.write(str(el)+'\n')
f.close()

time: 3.18 s (started: 2022-12-07 17:04:42 +00:00)


In [159]:
my_file = open("list_words_baskets.txt", "r")
list_loaded = my_file.readlines()
list_words_baskets = [eval(el) for el in list_loaded]

time: 46.7 ms (started: 2022-12-07 17:05:47 +00:00)


In [174]:
list_words_baskets

[['borisjohnson',
  'unfortunately',
  'boris',
  'government',
  'friend',
  'ally',
  'greatbritain',
  'europe',
  'witness'],
 ['raise',
  'hand',
  'want',
  'biden',
  'nato',
  'stand',
  'putin',
  'establish',
  'humanitarian',
  'fly',
  'zone',
  'istandwithukraine',
  'stoprussia',
  'eu',
  'russiaukrainewar'],
 ['brutal',
  'beat',
  'protest',
  'woman',
  'police',
  'man',
  'either',
  'work',
  'army',
  'film',
  'course',
  'shame',
  'httpstcohuxrgwlkps'],
 ['dmytrokuleba',
  'uaparliament',
  'europe',
  'amp',
  'nato',
  'close',
  'sky',
  'hear',
  'many',
  'innocent',
  'life',
  'child',
  'give',
  'away',
  'war',
  'emmanuelmacron',
  'vonderleyen',
  'eucopresident',
  'epedinburgh',
  'eucommission',
  'europarlen',
  'un',
  'potus',
  'olafscholz',
  'borisjohnson'],
 ['nytimes',
  'lynsey',
  'amazing',
  'shes',
  'kind',
  'person',
  'legend',
  'make',
  'love',
  'feel',
  'horrible',
  'family',
  'kill',
  'putin',
  'must',
  'pay',
  'stan

time: 166 ms (started: 2022-12-07 17:18:38 +00:00)


In [162]:
indices = range(0,899)
try_set = [list_words_baskets[i] for i in indices]
len(try_set)

899

time: 5.45 ms (started: 2022-12-07 17:09:28 +00:00)


In [ ]:
#Apriori Algoritm

In [163]:
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import numpy as np
import time

time: 1.16 ms (started: 2022-12-07 17:09:31 +00:00)


In [165]:
th_res = [0.01,0.02,0.06,0.09,0.1]
apriori_time_res = {}
for th in th_res:
    te = TransactionEncoder()
    oht_ary = te.fit(try_set).transform(try_set, sparse=True)
    sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
    start_time = time.time()
    res= apriori(sparse_df, min_support=th, use_colnames=True)
    apriori_time_res[th] = time.time() - start_time

time: 2.86 s (started: 2022-12-07 17:11:47 +00:00)


In [166]:
apriori_time_res

{0.01: 0.8311581611633301,
 0.02: 0.377866268157959,
 0.06: 0.2899489402770996,
 0.09: 0.29557371139526367,
 0.1: 0.2792367935180664}

time: 4.26 ms (started: 2022-12-07 17:12:04 +00:00)


In [167]:
ap_th_res = pd.DataFrame.from_dict(apriori_time_res, orient="index").reset_index()

new_column_list= ["Threshold","Time Elapsed"]

ap_th_res = ap_th_res.set_axis(new_column_list, axis=1)
ap_th_res

,Threshold,Time Elapsed
0,0.01,0.831158
1,0.02,0.377866
2,0.06,0.289949
3,0.09,0.295574
4,0.10,0.279237


time: 15.2 ms (started: 2022-12-07 17:12:25 +00:00)


In [170]:
te = TransactionEncoder()
oht_ary = te.fit(try_set).transform(try_set, sparse=True)
sparse_df = pd.DataFrame.sparse.from_spmatrix(oht_ary, columns=te.columns_)
res= apriori(sparse_df, min_support=0.02, use_colnames=True)

time: 584 ms (started: 2022-12-07 17:13:18 +00:00)


In [171]:
res.sort_values(by=['support'],ascending=False).head(10)

,support,itemsets
58,0.196885,(putin)
96,0.156841,(war)
74,0.154616,(standwithukraine)
90,0.121246,(ukrainewar)
53,0.092325,(people)
47,0.085651,(nato)
1,0.076752,(amp)
101,0.064516,(world)
66,0.063404,(russiaukrainewar)
68,0.062291,(say)


time: 16.1 ms (started: 2022-12-07 17:13:41 +00:00)
